In [ ]:
import glob
import mne 
import pandas as pd
import numpy as np
import seaborn as sns
import pingouin as pg
import statsmodels.api as sm
from statsmodels.formula.api import ols

#### Get evoked data

In [ ]:
evokeds_files = sorted(glob.glob('./analysis/Rise*-ave.fif'))
evks = []

for subj in evokeds_files:
    evks.append(mne.read_evokeds(subj, condition=None, verbose=False))

neutral_combined = mne.combine_evoked([e[1] for e in evks], weights='nave')
rise_combined = mne.combine_evoked([e[2] for e in evks], weights='nave')
fall_combined = mne.combine_evoked([e[3] for e in evks], weights='nave')

all_conds = [neutral_combined, rise_combined, fall_combined]

#### Rename channels

In [ ]:
# Replace all channels with proper channel names instead of numbers
channel_names_old = neutral_combined.ch_names
channel_names_new = ['Fp1','Fz','F3','F7','FT9','FC5','FC1','C3','T7','TP9','CP5','CP1','Pz','P3','P7','O1','Oz','O2','P4','P8','TP10','CP6',
                        'CP2','C4','T8','FT10','FC6','FC2','F4','F8','Fp2', 'AF7','AF3','AFz','F1','F5','FT7','FC3','C1','C5','TP7','CP3','P1','P5',
                        'PO7','PO3','POz','PO4','PO8','P6','P2','CPz','CP4','TP8','C6','C2','FC4','FT8','F6','AF8','AF4','F2','FCz', 'Cz']
channel_dict = dict(zip(channel_names_old, channel_names_new))

for cond in all_conds:
    mne.rename_channels(cond.info, mapping=channel_dict)

#### Peak Latency & Amplitude

In [ ]:
# Function to print out the channel (ch) containing the peak latency and amplitude within the provided time range 
def print_peak_measures(ch, tmin, tmax, lat, amp):
    print(f'Channel: {ch}')
    print(f'Time Window: {tmin * 1e3:.3f} - {tmax * 1e3:.3f} ms')
    print(f'Peak Latency: {lat * 1e3:.3f} ms')
    print(f'Peak Amplitude: {amp * 1e6:.3f} µV')


# Get peak amplitude and latency from a good time window that contains the peak
def get_peak_measures(cond, ch_type, tmin, tmax):
    ch, lat, amp = cond.get_peak(ch_type=ch_type, tmin=tmin, tmax=tmax, mode='abs', return_amplitude=True)
    print_peak_measures(ch, tmin, tmax, lat, amp)

#### Compare P300 amplitudes & latencies (Pz and CPz)

In [ ]:
chs = ['Pz', 'CPz']

for evk in all_conds:
    evk.plot(picks=chs, spatial_colors=True, highlight=(200, 400), time_unit='ms', ylim=dict(eeg=[-1.5, 2]))

#### Compare Rise & Fall (combined P3, P5, CPz, Pz)

In [ ]:
mne.viz.plot_compare_evokeds(all_conds[1: ], picks=['P3', 'P5', 'CPz', 'Pz'], combine='mean', ci=0.95, legend=False)